# DASK DP hyperparameter selection

Note: This notebook is intended to be run on the DASK scheduler.

The cross-validation structure is quite complicated, to explains;

- We split the data into training/test sets (normal cross-validation); for each split we want the probability of selecting each parameter configuration, and the score if that parameter selection is used.
- The probability of selecting that configuration is computed purely from the training data (to avoid peeking) [see lines 172-175 for the relevant calls]. Within each training data set two seperate calls are made to GridSearchCV. The first [line 109] gets the sensitivities of each fold/parameter-set of the SSE. The second [line 120] gets the negative SSE of each fold/parameter-set. We can then use these two to compute probabilities of selecting each parameter configuration using the exponential mechanism.
- The RMSE is computed on the same train/test splits (but in a seperate call, [on line 142]).
- At the end of the computation we are left with RMSEs and probabilities, we are then able to find the unbias expected RMSE.

### TODO

- Test and check all this code.
- Add unnormalisation step at end.
- Currently passing kernel objects as points in the parameter grid causes a pickle error inside DASK. Instead I've added parameters to the class to allow lengthscale and variance to be changed. Ideally however we want to have a kernel passed to allow this class to be more flexible.

In [ ]:
## Install libraries if necessary on workers etc.

from dask.distributed import Client
client = Client('127.0.0.1:8786')
import os
import distributed

runlist = ['pip install -U pip','sudo apt install libgl1-mesa-glx -y','conda update scipy -y','pip install git+https://github.com/sods/paramz.git','pip install git+https://github.com/SheffieldML/GPy.git','pip install git+https://github.com/lionfish0/dp4gp.git','conda install dask-searchcv -c conda-forge -y']

for item in runlist:
    print("Installing '%s' on workers..." % item)
    client.run(os.system,item)
    print("Installing '%s' on scheduler..." % item)
    client.run_on_scheduler(os.system,item)    
    #os.system(item) #if you need to install it locally too

In [3]:
from dp4gp import datasets
from dp4gp import dp4gp
from dp4gp.utils import dp_normalise, dp_unnormalise
import random
import numpy as np
import GPy
import matplotlib.pyplot as plt
from dp4gp import histogram
import pandas as pd
from dask_searchcv import GridSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score # http://scikit-learn.org/stable/developers/contributing.html#estimators
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.cluster import KMeans
%matplotlib inline

#from dask.distributed import Client
#client = Client('127.0.0.1:8786')

kung = datasets.load_kung()

# This is an estimator for sklearn
class DPCloaking(BaseEstimator):
    def __init__(self, lengthscale=None, variance=None, errorlimit=None, kern=None, sensitivity=1.0, epsilon=1.0, delta=0.01, inducing=None, noisevariance=None, getxvalfoldsensitivities=False):
        """
        DPCloaking(lengthscale=None, variance=None, errorlimit=None, kern=None, sensitivity=1.0, epsilon=1.0, delta=0.01, inducing=None, noisevariance=None, getxvalfoldsensitivities=False)
        
        lengthscale=None - kernel lengthscale
        variance=None - kernel variance
        noisevariance=None - model gaussian white noise
        errorlimit=None - when using the class' score function to report the SSE
                we can set a limit on how large a single error is allowed to be.
        kern=None - a GPy kernel (needs to have lengthscales and variances)
        sensitivity=1.0 - the amount one output can change
        epsilon=1.0, delta=0.01 - DP parameters
        inducing = None - locations of inducing points, default to None - not using inducing points.
        getxvalfoldsensitivities=False - the score method will return the sum-squared-error
                of each fold by default, but if this is set to true it will return the
                sensitivity of the SSE caused by a perturbed point being in the training data. 
        """
        
        self.errorlimit = errorlimit
        self.kern = kern
        self.sensitivity = sensitivity
        self.epsilon = epsilon
        self.delta = delta
        self.inducing = inducing
        self.getxvalfoldsensitivities = getxvalfoldsensitivities

        if lengthscale is not None:
            self.kern.lengthscale = lengthscale
        if variance is not None:
            self.kern.variance = variance
        if noisevariance is not None:
            self.noisevariance = noisevariance
        else:
            self.noisevariance = 1.0
        
    def fit(self, X, y, **kwargs):
        """
        fit(X, y)
        
        Create the GPy model using the data in X and y, and the hyperparameters etc set by
        the constructor (or subsequently during a grid search)
        """
        self.kern.lengthscale = self.lengthscale
        self.kern.variance = self.variance
        if self.inducing is None:
            self.model = GPy.models.GPRegression(X,y,self.kern,normalizer=None)
            self.model.Gaussian_noise = self.noisevariance
            self.dpgp = dp4gp.DPGP_cloaking(self.model,self.sensitivity,self.epsilon,self.delta)
        else:
            if isinstance(self.inducing, list):
                inducinglocs = self.inducing
            else:
                inducinglocs = KMeans(n_clusters=self.inducing, random_state=0).fit(X).cluster_centers_
            self.model = GPy.models.SparseGPRegression(X,y,self.kern,normalizer=None,Z=inducinglocs)
            self.model.Gaussian_noise = self.noisevariance
            self.dpgp = dp4gp.DPGP_inducing_cloaking(self.model,self.sensitivity,self.epsilon,self.delta)
        return self

    def predict(self, X, Nattempts=2, Nits=5): #todo set Nits back to a larger value (e.g. 100)
        """
        predict(X,Nattempts=2, Nits=100)
        
        make predictions of the outputs, y, given inputs X.
        """
        ypred,_,_= self.dpgp.draw_prediction_samples(X,Nattempts=Nattempts,Nits=Nits)
        return ypred
    
    def score(self, X, y=None):
        """
        score(X,y=None)
        
        Return the (truncated) negative SSE or part of the sensitivity of the SSE
        
        If self.getxvalfoldsensitivities is True then, this isn't the actual score.
        Instead it returns the sensitivity of the sum squared error if the perturbed
        point lies in the training data.
        
        If it's false this returns the truncated NEGATIVE Sum Squared Error
        
            Note on truncation: we have also put a bound on the error of
            one data point - it must lie within errorlimit of the actual
            prediction. If it doesn't then the value is truncated.

            Note: This score is used to select the hyperparameter config
            used later in the cross-validation procedure. The truncation
            will therefore not cause an underestimate in the final score.
            The truncation allows us to put a bound on the sensitivity of
            the SSE.
        """        
        if self.getxvalfoldsensitivities:          
            C = self.dpgp.get_C(X) #NumTest x NumTrain
            #we sum over the square of this (times the sensitivity) to get,
            #             |d c_kj|_2^2
            #we then find the largest value of this to get,
            #             d^2 max_j |c_kj|_2^2
            #and this we return, giving the largest effect a training
            #point can have on the SSE.
            return np.max(np.sum((self.sensitivity * C)**2,0))
        else:          
            errors = (y-self.predict(X))
            errors[errors>self.errorlimit] = self.errorlimit
            errors[errors<-self.errorlimit] = -self.errorlimit
            return -np.sum((y-self.predict(X))**2)

def getprobabilities(X,y,p_grid, cv):
    """
    getprobabilities(X,y,p_grid, cv)
    
    - X and y: Inputs and outputs
    - p_grid: grid of parameters to search over
    - cv: Number of cross-validation folds (this is different from the outer number of x-val folds)
    
    Gets the probability of picking each of the options provided by p_grid given the data in X and y.
    The algorithm is as follows:
     - Find the sensitivity of the SSE for each parameter-values
     - Find the SSE of each parameter-values
     - Find the probability of selecting those parameter-values
    """
    kern = GPy.kern.RBF(2.0,lengthscale=25.0,variance=1.0)
    errorlimit = ac_sens*4.0
   
    ####find sensitivity of the SSE (for each param combo)
    #this call gets the sensivities not the scores:
    #TODO This probably should be done locally as it's quick.
    clf = GridSearchCV(estimator=DPCloaking(sensitivity=ac_sens, inducing=4, getxvalfoldsensitivities=True, kern=kern, errorlimit=errorlimit), param_grid=p_grid, cv = cv)
    clf.fit(X,y)

    nparamcombos = len(clf.cv_results_['mean_test_score'])
    temp_sens = np.zeros([clf.cv,nparamcombos])
    for k in range(clf.cv):
        temp_sens[k,:] = clf.cv_results_['split%d_test_score' % k]
    #sensitivity of the sum squared error:
    print(np.sort(temp_sens,axis=0))
    sse_sens = ac_sens**2 + 2*ac_sens*errorlimit + ac_sens**2*np.max(np.sum(np.sort(temp_sens,axis=0)[0:clf.cv-1,:],0))

    ####find the SSE (for each param combo)
    clf = GridSearchCV(estimator=DPCloaking(sensitivity=ac_sens, inducing=4, getxvalfoldsensitivities=False, kern=kern, errorlimit=errorlimit), param_grid=p_grid, cv = cv)
    clf.fit(X,y)

    nparamcombos = len(clf.cv_results_['mean_test_score'])
    temp_scores = np.zeros([clf.cv,nparamcombos])
    for k in range(clf.cv):
        temp_scores[k,:] = clf.cv_results_['split%d_test_score' % k]
    scores = np.sum(temp_scores,0)

    ####compute the probability of selecting that param combo using the exponential mechanism
    selection_epsilon = 1
    param_probabilities = np.exp(selection_epsilon*scores / (2*sse_sens))
    param_probabilities = param_probabilities / np.sum(param_probabilities)
    
    return param_probabilities

def getscores(X,y,p_grid,cv):
    """
    Compute the negative RMSE of each of the fold/param combos
    """
    kern = GPy.kern.RBF(2.0,lengthscale=25.0,variance=1.0)

    clf = GridSearchCV(estimator=DPCloaking(sensitivity=ac_sens, inducing=4, getxvalfoldsensitivities=False, kern=kern), scoring='neg_mean_squared_error', param_grid=p_grid, cv = cv)
    clf.fit(X,y)

    nparamcombos = len(clf.cv_results_['mean_test_score'])
    scores = np.zeros([clf.cv,nparamcombos])
    for k in range(clf.cv):
        scores[k,:] = clf.cv_results_['split%d_test_score' % k]
    return scores

####Set up data and parameter search grid
sensitivity = 100.0
y,ac_sens,norm_params = dp_normalise(kung[kung[:,3]==0,0:1],sensitivity)
X = kung[kung[:,3]==0,1:3]
epsilon = 1.0
delta = 0.01
cv = 3
p_grid = {"lengthscale":[], 'variance':[]}#, 'noisevariance':[]}
for ls in 5.0**np.arange(0,2):
    p_grid["lengthscale"].append(ls)
for v in 5.0**np.arange(-1,1):
    p_grid["variance"].append(v)
    
####Get the -RMSE for each fold/param-combo
scores = getscores(X,y,p_grid,cv)

from sklearn.model_selection import KFold


kf = KFold(n_splits=cv)
probabilities = []
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    y_train = y[train_index]
    probabilities.append(getprobabilities(X_train,y_train,p_grid,5))
    
print(np.array(probabilities))
print(scores)
print(np.sum(probabilities*-scores,1))
print(np.mean(-scores,1))

 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike

[[ 0.0042492   0.02870927  0.2367837   0.69135983]
 [ 0.02662131  0.20103365  0.24542001  0.69299734]
 [ 0.048709    0.43837831  0.33126626  0.84603282]
 [ 0.08270285  0.48933589  0.38952283  1.31397035]
 [ 0.19239383  1.25068521  0.42003588  1.81254158]]


 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Docume

[[  1.04379140e-03   9.39256749e-03   1.46038836e-01   5.91199594e-01]
 [  3.18851435e-02   2.20208406e-01   2.07628670e-01   6.29649828e-01]
 [  4.04117234e-02   3.34054795e-01   2.47510204e-01   7.13719276e-01]
 [  8.87866119e-02   7.84130685e-01   2.83409834e-01   1.04308255e+00]
 [  2.12443756e-01   1.12550033e+00   3.40077077e-01   1.86990344e+00]]


 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Docume

[[  2.48378852e-04   1.82724148e-03   1.72865198e-01   5.04249809e-01]
 [  1.66914053e-02   1.26795300e-01   2.67845278e-01   6.64986382e-01]
 [  2.03271379e-02   1.66820512e-01   3.02653537e-01   7.56399727e-01]
 [  5.73889258e-02   3.80958199e-01   3.02935611e-01   9.89294247e-01]
 [  1.88143169e-01   1.21890455e+00   5.18963556e-01   2.39145729e+00]]


 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:418: RuntimeWarning:covariance is not positive-semidefinite.
 /home/mike/Documents/Research/dp4gp/dp4gp/dp4gp.py:343: RuntimeWarning:divide by zero encountered in log
 /home/mike/Docume

[[ 0.25212614  0.23566723  0.29164709  0.22055954]
 [ 0.26988541  0.2265527   0.27359939  0.2299625 ]
 [ 0.31079512  0.23076745  0.29491875  0.16351868]]
[[-0.5219669  -0.43615581 -0.26681739 -0.73298314]
 [-0.66548054 -0.77083157 -0.63909182 -0.4343488 ]
 [-1.83251219 -1.81390514 -2.01421716 -1.87843712]]
[ 0.47387207  0.62897653  1.88931607]
[ 0.48948081  0.62743818  1.8847679 ]


In [3]:
scores

array([[-0.60348814, -0.50939675, -0.65877968, -0.27725199],
       [-0.65522194, -0.95756242, -0.58511046, -0.98094521],
       [-1.8343872 , -1.9017283 , -1.55337318, -2.03818546]])

In [4]:
probabilities

[array([ 0.2780088 ,  0.2651039 ,  0.25994567,  0.19694163]),
 array([ 0.26712682,  0.23700197,  0.31964679,  0.17622442]),
 array([ 0.30261986,  0.27097601,  0.26606621,  0.16033791])]